<a id='top'></a>
# calwebb_detector1 run stage
---
**Author**: Jonathan Aguilar (jaguilar@stsci.edu) | **Latest Update**: 8 Nov 2023

* [Introduction](#intro)
* [Non-pipeline imports](#imports)
* [Convenience tools](#convenience_tools)
* [Collect uncal files](#file_selection)
* [Run Stage 1](#detector1)

<a id='intro'></a>
## Introduction

This notebook run the Stage 1 pipeline on all the relevant reference science, PSF and background files for the JWST-ERS-1386 observations of HIP 65426 with the F1550C coronagraph. The relevant observation numbers are:

- Science observations: 8, 9
- Science backgrounds: 30
- Reference observations: 7
- Reference backgrounds: 31

For more details, especially on how to customize the individual pipeline steps, see `calwebb-detector1-single_exposure.ipynb` as well as the online pipeline documentation and examples:
- https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html#calwebb-detector1
- https://github.com/spacetelescope/jwebbinar_prep/blob/main/pipeline_inflight/imaging_mode_stage_1.ipynb

Here we will simply be using default parameters, except that the output directory will be set to `./stage1/output/`

<a id='imports'></a>
## Imports

In [ ]:
import os
from collections import OrderedDict
from pathlib import Path

In [ ]:
import numpy as np

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
from astropy.io import fits

<a id='convenience_tools'></a>
## Convenience tools

Environment paths and functions that make life easier.

First, set up a local CRDS directory. When the pipeline pulls a reference file from CRDS for the first time, it will write a copy to this directory. All subsequent reads of the reference file will redirect to the local directory instead of sending the file again over the network.

See https://jwst-pipeline.readthedocs.io/en/latest/jwst/user_documentation/reference_files_crds.html#crds

In [ ]:
os.environ['CRDS_PATH'] = '/Volumes/agdisk/crds/'
# os.environ['CRDS_PATH'] = ''
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

Advanced users - uncomment the cell below and specify the context if you have a specific combination of reference files you want to use

In [ ]:
# os.environ['CRDS_CONTEXT'] = 'jwst_1140.pmap'

In [ ]:
# some plot formatting
mpl.rcParams['image.origin'] = "lower"

In [ ]:
# you may need to run this command twice for plots to pop up correctly
# %matplotlib auto
%matplotlib inline

In [ ]:
# this helper function to map out a particular DQ flag will be useful
# DQ flags can be found here: 
# https://jwst.readthedocs.io/en/latest/jwst/references_general/references_general.html#data-quality-flags
def get_dq_flag(flag, dq_img):
    """return the pixels that have a given DQ flag"""
    bad_bitvalue = dqflags.pixel[flag]
    flags = np.bitwise_and(dq_img, bad_bitvalue).astype(bool)
    return flags

<a id="file_selection"></a>
## Collect uncal files

The cell below will access MAST to retrieve the relevant exposures from Observations 4, 5, 6, 28, and 29 of the High-contrast ERS program, PID 1386. 

If you would like to use different exposures, please replace the `filenames` variable with a list of paths to those files.

In [ ]:
filenames = [
    'jw01386008001_04101_00001_mirimage_uncal.fits',  # Science target, roll 1
    'jw01386009001_04101_00001_mirimage_uncal.fits',  # Science target, roll 2
    'jw01386007001_04101_00001_mirimage_uncal.fits',  # PSF reference star observations
    'jw01386007001_04101_00002_mirimage_uncal.fits',
    'jw01386007001_04101_00003_mirimage_uncal.fits',
    'jw01386007001_04101_00004_mirimage_uncal.fits',
    'jw01386007001_04101_00005_mirimage_uncal.fits',
    'jw01386007001_04101_00006_mirimage_uncal.fits',
    'jw01386007001_04101_00007_mirimage_uncal.fits',
    'jw01386007001_04101_00008_mirimage_uncal.fits',
    'jw01386007001_04101_00009_mirimage_uncal.fits',
    'jw01386030001_02101_00001_mirimage_uncal.fits',  # Science backgrounds
    'jw01386030001_03101_00001_mirimage_uncal.fits',
    'jw01386031001_02101_00001_mirimage_uncal.fits',  # PSF reference star backgrounds
    'jw01386031001_03101_00001_mirimage_uncal.fits',
]
from astroquery.mast import Observations
for filename in filenames:
    Observations.download_file(f"mast:JWST/product/{filename}", local_path= f"./uncal/{filename}")
uncal_file = f"./uncal/{filenames[0]}"

Let's do a quick inspection of one of the files

In [ ]:
# print some basic information
fits.info(uncal_file)

In [ ]:
# a simple plot
fig, ax = plt.subplots(1, 1)
img = np.nanmean(fits.getdata(uncal_file, 1), axis=(0, 1))
imlims = dict(zip(['vmin', 'vmax'], np.nanquantile(img, [0.1, 0.9])))
ax.imshow(img, **imlims, origin='lower')

<a id='detector1'></a>

## Run calwebb_detector1 on each file

The command `Detector1Pipeline().get_pars()` will return a dictionary listing all the available parameters and their default values.

This dictionary can be modified and then used with the commend `Detector1Pipeline.call(filename, **parameter_dict)`.

In [ ]:
import jwst
jwst.__version__

In [ ]:
from jwst.pipeline import Detector1Pipeline

In [ ]:
for i, f in enumerate(filenames):
    # run det1
    print("\n\n\n", i+1, len(filenames), "\n\n\n")
    filename = str(Path("uncal") / f)
    output_dir = "stage1/output/"
    res1 = Detector1Pipeline.call(
        filename,
        output_dir=output_dir,
        save_results=True
        steps={
            'jump': {'maximum_cores': 'half'},
            'ramp_fit': {'maximum_cores': 'half'}
        }
    )

The results are stored in `./stage1/output/`.